# Examine what effect the number of ensemble members has

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.cm as cm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.patheffects as pe
import xarray as xr
import datetime
import dask
print(datetime.datetime.utcnow().strftime("%H:%M UTC %a %Y-%m-%d"))

19:27 UTC Fri 2021-10-15


In [2]:
path = '/glade/scratch/cwpowell/Synthetic_ensemble/'

month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 'November', 'December']

model_names        = ['CanESM2', 'CESM1', 'CSIRO_MK36',  'GFDL_CM3', 'GFDL_ESM2M', 'MPI_ESM1']
model_print_names  = ['CanESM2', 'CESM1', 'CSIRO MK3.6', 'GFDL CM3', 'GFDL ESM2M', 'MPI ESM1']
obs_names          = ['HadISST1']
mem_len   = [50,  40,  30,  20,  30,  100]
start_yrs = [1950,1920,1850,1920,1950,1850]
colors    = ['m', 'b', 'g', 'orange', 'k', 'tab:olive']

# Subsample the members

In [3]:
#load detrended data
CLIVAR_det_ind = xr.open_dataset(path+'SIA/SIA_detrended/CLIVAR_SIA_detrended_individual_79-20.nc')
CLIVAR_det_ens = xr.open_dataset(path+'SIA/SIA_detrended/CLIVAR_SIA_detrended_ensemble_79-20.nc')

#compute the standard deviation with respect to time for each month
ind_SD = CLIVAR_det_ind.groupby('time.month').std('time')
ens_SD = CLIVAR_det_ens.groupby('time.month').std('time')

## $\sigma_{LE}$

In [5]:
def bootstrap_month_model(sd_time, mem_n, resamp_n):
    
    SD_LE = []
    
    for mem_n_i in np.arange(2,mem_n+1,1): #loop through 2 the maximum number of members

        temp_SD_list = [] #initialize array to record all the resamplings

        for resamp_i in range(resamp_n):
            rand_i_list = np.random.randint(0,mem_n-1, size=mem_n_i) #make a list of random numbers between 0 and last member element
            temp_SD_list.append(sd_time.isel(member=rand_i_list).std('member'))

        SD_LE.append(np.mean(temp_SD_list))

    #make an xarray object with properly named coordinates for members
    SD_LE = xr.DataArray(data   = SD_LE,
                         coords = {'n_members':np.arange(2,mem_n+1,1)},
                         dims   = ['n_members'])
    
    return(SD_LE)

In [6]:
boostrapped_ind = []
# boostrapped_ens = []

for month_ in [3,9]:
    print(datetime.datetime.now(), month_)
    month_ind = []
#     month_ens = []
    
    for model_i, model_name in enumerate(model_names):
        month_ind.append(bootstrap_month_model(ind_SD[model_name].sel(month=month_), mem_len[model_i], 1000))
#         month_ens.append(bootstrap_month_model(ens_SD[model_name].sel(month=month_), mem_len[model_i], 1000))
        
    boostrapped_ind.append(xr.concat((month_ind), dim='model'))
#     boostrapped_ens.append(xr.concat((month_ens), dim='model'))
    
boostrapped_ind = xr.concat((boostrapped_ind), dim='month')
boostrapped_ind['month'] = [3,9]
boostrapped_ind['model'] = model_names

# boostrapped_ens = xr.concat((boostrapped_ens), dim='month')
# boostrapped_ens['month'] = [3,9]
# boostrapped_ens['model'] = model_names

2021-10-15 13:28:22.417314 3
2021-10-15 13:30:35.158773 9


## $\sigma_{mem}$

In [9]:
#load sigma_mem
sigma_mem = xr.open_dataset(path+'SIA/SIA_resampled/Sigma_mem_individual_10000.nc')

In [16]:
boostrapped_mem = []

for month_ in [3,9]:
    print(datetime.datetime.now(), month_)
    month_mem = []
    
    for model_i, model_name in enumerate(model_names):
        print(datetime.datetime.now(), model_name)
        month_mem.append(bootstrap_month_model(sigma_mem[model_name].sel(month=month_), mem_len[model_i], 1000))
        
    boostrapped_mem.append(xr.concat((month_mem), dim='model'))
    
boostrapped_mem = xr.concat((boostrapped_mem), dim='month')
boostrapped_mem['month'] = [3,9]
boostrapped_mem['model'] = model_names

In [15]:
boostrapped_mem = xr.concat((boostrapped_mem), dim='month')
boostrapped_mem['month'] = [3,9]
boostrapped_mem['model'] = model_names

# Plot
## $\sigma_{LE}$

In [28]:
#1979-2020
#open the dataset
data = boostrapped_ind.copy() #change for ind or ens

fig, axes = plt.subplots(2,2,figsize=[15,8])

#plot the sigma LE with number of members
for i, month_ in enumerate([3,9]):
    
    for model_i, model_name in enumerate(model_names):
        axes[i][0].plot(np.arange(2,101,1), data.sel(model=model_name).sel(month=month_), c=colors[model_i], linewidth=2.5)
        
    axes[i][0].set_ylim(0.018,0.086) #max 0.083 for ind, 0.086 for ens
    axes[i][0].set_ylabel(r'$\sigma_{LE} \ SIA \ [10^6 \ km^2]$', fontsize=20);
    axes[i][0].set_yticklabels(np.round(np.arange(0.01,0.09,0.01),2), fontsize=16)
    axes[i][0].set_xlim(2,51)
    axes[i][0].set_xticks(np.arange(5,51,5))
    axes[i][0].set_xticklabels(np.round(np.arange(5,51,5)), fontsize=16)
    axes[i][0].grid()

    
    for model_i, model_name in enumerate(model_names):
        axes[i][1].plot(np.arange(2,101,1), np.gradient(data.sel(model=model_name).sel(month=month_)), c=colors[model_i], linewidth=2.5)
        
    axes[i][1].axhline(0, c='0.5', linewidth=0.25)
    axes[i][1].set_ylabel(r'$\sigma_{LE}$  Derivative', fontsize=20);
    axes[i][1].set_ylim(-0.0005,0.013)
    axes[i][1].set_yticks(np.arange(0,0.013,0.002))
    axes[i][1].set_yticklabels(['0', '0.002', '0.004', '0.006', '0.008', '0.010', '0.012'], fontsize=16)
    axes[i][1].set_xlim(2,20)
    axes[i][1].set_xticks(np.arange(2,21,2))    
    axes[i][1].set_xticklabels(np.round(np.arange(2,21,2)), fontsize=16)   
    axes[i][1].grid()
    
legend_elements = [Patch(facecolor=colors[0], label='CanESM2'),
                   Patch(facecolor=colors[1], label='CESM1'),
                   Patch(facecolor=colors[2], label='CSIRO MK36'),
                   Patch(facecolor=colors[3], label='GFDL CM3'),
                   Patch(facecolor=colors[4], label='GFDL ESM2M'),
                   Patch(facecolor=colors[5], label='MPI ESM1'),]

# extra_legend = plt.legend(handles=legend_elements, bbox_to_anchor=(0.85, 0.45), loc='lower center', borderaxespad=0, ncol=1, fontsize=14)
# plt.gca().add_artist(extra_legend);

extra_legend = plt.legend(handles=legend_elements, bbox_to_anchor=(-0.11, -0.41), loc='lower center', borderaxespad=0, ncol=6, fontsize=16)
plt.gca().add_artist(extra_legend);

axes[1][0].set_xlabel('Number of members', fontsize=20)
axes[1][1].set_xlabel('Number of members', fontsize=20)
axes[0][0].text(-0.06, 1.08, 'a', transform=axes[0][0].transAxes, fontsize=20, fontweight='bold', va='top', ha='right');
axes[0][1].text(-0.06, 1.08, 'b', transform=axes[0][1].transAxes, fontsize=20, fontweight='bold', va='top', ha='right');
axes[1][0].text(-0.06, 1.08, 'c', transform=axes[1][0].transAxes, fontsize=20, fontweight='bold', va='top', ha='right');
axes[1][1].text(-0.06, 1.08, 'd', transform=axes[1][1].transAxes, fontsize=20, fontweight='bold', va='top', ha='right');

plt.tight_layout()
fig.subplots_adjust(bottom=0.2)
fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIA/figures/Member_number_and_sigma/Bootstrapping_ind_larger.pdf', bbox_inches='tight') #change file name for ind or ens
fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIA/figures/Member_number_and_sigma/Bootstrapping_ind_larger.png', 
            dpi=400, bbox_inches='tight')

## $\sigma_{mem}$

In [27]:
#1979-2020
#open the dataset
data = boostrapped_mem.copy() 

fig, axes = plt.subplots(2,2,figsize=[15,8])

#plot the sigma LE with number of members
for i, month_ in enumerate([3,9]):
    
    for model_i, model_name in enumerate(model_names):
        axes[i][0].plot(np.arange(2,101,1), data.sel(model=model_name).sel(month=month_), c=colors[model_i], linewidth=2.5)
        
    axes[i][0].set_ylim(0.002,0.019) #max 0.083 for ind, 0.086 for ens
    axes[i][0].set_ylabel(r'$\sigma_{mem} \ SIA \ [10^6 \ km^2]$', fontsize=14);
    axes[i][0].set_yticklabels(np.round(np.arange(0.002,0.019,0.002),4), fontsize=12)
    axes[i][0].set_xlim(2,51)
    axes[i][0].set_xticks(np.arange(5,51,5))
    axes[i][0].set_xticklabels(np.round(np.arange(5,51,5)), fontsize=12)
    axes[i][0].grid()

    
    for model_i, model_name in enumerate(model_names):
        axes[i][1].plot(np.arange(2,101,1), np.gradient(data.sel(model=model_name).sel(month=month_)), c=colors[model_i], linewidth=2.5)
        
    axes[i][1].axhline(0, c='0.5', linewidth=0.25)
    axes[i][1].set_ylabel(r'$\frac{d \ \sigma_{mem}}{d \ member} \ SIA \ [10^6 \ km^2 \ member^{-1}]$', fontsize=14);
    axes[i][1].set_ylim(-0.0002,0.0033)
    axes[i][1].set_yticks(np.arange(0,0.0031,0.0005))
    axes[i][1].set_yticklabels(['0', '0.0005', '0.0010', '0.0015', '0.0020', '0.0025', '0.0030'], fontsize=12)
    axes[i][1].set_xlim(2,20)
    axes[i][1].set_xticks(np.arange(2,21,2))    
    axes[i][1].set_xticklabels(np.round(np.arange(2,21,2)), fontsize=12)   
    axes[i][1].grid()
    
legend_elements = [Patch(facecolor=colors[0], label='CanESM2'),
                   Patch(facecolor=colors[1], label='CESM1'),
                   Patch(facecolor=colors[2], label='CSIRO MK36'),
                   Patch(facecolor=colors[3], label='GFDL CM3'),
                   Patch(facecolor=colors[4], label='GFDL ESM2M'),
                   Patch(facecolor=colors[5], label='MPI ESM1'),]

extra_legend = plt.legend(handles=legend_elements, bbox_to_anchor=(0.85, 0.45), loc='lower center', borderaxespad=0, ncol=1, fontsize=12)
plt.gca().add_artist(extra_legend);

axes[1][0].set_xlabel('Number of members', fontsize=14)
axes[1][1].set_xlabel('Number of members', fontsize=14)
axes[0][0].text(-0.06, 1.08, 'a', transform=axes[0][0].transAxes, fontsize=18, fontweight='bold', va='top', ha='right');
axes[0][1].text(-0.06, 1.08, 'b', transform=axes[0][1].transAxes, fontsize=18, fontweight='bold', va='top', ha='right');
axes[1][0].text(-0.06, 1.08, 'c', transform=axes[1][0].transAxes, fontsize=18, fontweight='bold', va='top', ha='right');
axes[1][1].text(-0.06, 1.08, 'd', transform=axes[1][1].transAxes, fontsize=18, fontweight='bold', va='top', ha='right');

plt.tight_layout()
fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIA/figures/Member_number_and_sigma/Bootstrapping_mem_ind.pdf') #change file name for ind or ens
fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIA/figures/Member_number_and_sigma/Bootstrapping_mem_ind.png', dpi=400)